# Compute Trustworthiness Scores for any LLM

<head>
  <meta name="title" content="How to get trustworthiness scores for any LLM"/>
  <meta property="og:title" content="How to get trustworthiness scores for any LLM"/>
  <meta name="twitter:title" content="How to get trustworthiness scores for any LLM" />
  <meta name="image" content="/img/add-trust-llm.png" />
  <meta property="og:image" content="/img/add-trust-llm.png" />
  <meta name="description" content="Quantify Trustworthiness for every output and detect bad responses from any LLM."  />
  <meta property="og:description" content="Quantify Trustworthiness for every output and detect bad responses from any LLM." />
  <meta name="twitter:description" content="Quantify Trustworthiness for every output and detect bad responses from any LLM." />
</head>



Before this tutorial, we recommend first completing the [TLM quickstart tutorial](/tutorials/tlm/). 

Most of the TLM examples shown throughout this documentation (including the quickstart tutorial) demonstrate how to use Cleanlab's Trustworthy Language Model, which is powered by a predefined set of underlying LLMs you can choose from via the `model` key in TLMOptions.

TLM is actually a general-purpose wrapper technology that can make *any* LLM more trustworthy. This tutorial demonstrates **how to produce TLM trustworthiness scores for your own LLM**. Here we demonstrate this for [Ollama](https://ollama.com/), a open-source implementation of the [Llama-3](https://ai.meta.com/blog/meta-llama-3/) model you can run locally on your own laptop. You can replace Ollama with any other LLM and still follow this tutorial.

## Install and import dependencies

Using TLM requires a [Cleanlab account](https://app.cleanlab.ai/). Here we use Ollama (specifically the Llama-3 model) via the open-source [langchain-community](https://api.python.langchain.com/en/latest/community_api_reference.html) library. First go to [https://ollama.com/](https://ollama.com/) and follow the instructions to **download the Llama-3 model**.

The Python packages required for this tutorial can be installed using pip (we used `langchain-community` version 0.0.34):

In [1]:
%pip install --upgrade cleanlab-studio langchain-community

In [2]:
from langchain_community.llms import Ollama
from cleanlab_studio import Studio
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

Here's how you would regularly use your Ollama LLM:

In [3]:
my_llm = Ollama(model="llama3")

# Example prompt, feel free to ask other questions.
prompt = "What is the 3rd month if we list all the months of the year in alphabetical order?"

response = my_llm.invoke(prompt)

print(response)

If we list the 12 months of the year in alphabetical order, they are:

1. April
2. August
3. December
4. February
5. January
6. July
7. June
8. March
9. May
10. November
11. October
12. September

So, the 3rd month is actually "March"!


Here's how to compute the corresponding TLM trustworthiness score for this Llama-3 LLM response:

In [4]:
from cleanlab_studio import Studio

# Get API key from here: https://app.cleanlab.ai/account after creating an account.
studio = Studio("<API key>")

cleanlab_tlm = studio.TLM()  # see TLM quickstart tutorial for optional TLM configurations

trustworthiness_score = cleanlab_tlm.get_trustworthiness_score(prompt, response=response)

print(trustworthiness_score["trustworthiness_score"])

0.28222242491982585


Let's define an object we can prompt and get both the LLM response and associated trustworthiness score:

In [5]:
class TrustworthyLanguageModel:
    """ Class that returns responses from your LLM and associated trustworthiness scores. """
    def __init__(self, response_llm, score_tlm):
        self.response_llm = response_llm
        self.score_tlm = score_tlm

    def prompt(self, prompt: str):
        """ 
        Returns a dict with keys: 'response' and 'trustworthiness_score'
        where response is produced by your response_llm.
        This implementation assumes response_llm has a invoke(prompt) method.
        """
        output = {}
        output['response'] = self.response_llm.invoke(prompt)
        output['trustworthiness_score'] = self.score_tlm.get_trustworthiness_score(prompt, response=output['response'])["trustworthiness_score"]
        return output

    def prompt_batch(self, prompts: list[str]):
        """
        Version of prompt() where you can also pass in a list of many prompts
        and get lists of responses and trustworthiness scores back.
        This implementation assumes response_llm has a batch(prompts) method.
        """
        outputs = {}
        outputs['response'] = self.response_llm.batch(prompts)
        trustworthiness_scores = self.score_tlm.get_trustworthiness_score(prompts, response=outputs['response'])
        outputs['trustworthiness_score'] = [score['trustworthiness_score'] for score in trustworthiness_scores]
        return outputs


In [6]:
my_tlm = TrustworthyLanguageModel(my_llm, cleanlab_tlm)

output = my_tlm.prompt(prompt)

print(f"Response:  {output['response']} \n \n Trustworthiness:  {output['trustworthiness_score']}")

Response:  When you list all the months of the year in alphabetical order, they are:

1. April
2. August
3. December
4. February
5. January
6. July
7. June
8. March
9. May
10. November
11. October
12. September

So, the 3rd month in alphabetical order is... March! 
 
 Trustworthiness:  0.260353310016186


This allows you to easily obtain responses and associated trustworthiness scores for any LLM!

## Running our custom TLM over a dataset of prompts

Firstly let's load in an example query dataset. Consider a dataset of various Space-related trivia questions. We can use each question as a prompt for our custom TLM and record its response and associated trustworthiness score.

In [15]:
!wget -nc 'https://cleanlab-public.s3.amazonaws.com/Datasets/solar_system_dataset.csv'
!wget -nc 'https://cleanlab-public.s3.amazonaws.com/Datasets/solar_system_example_prompts.csv'

In [7]:
df = pd.read_csv("solar_system_example_prompts.csv")
df.head()

,prompt
0,What is the largest planet in the Solar System?
1,"As of 2024, how many dogs have reached outer space?"
2,What is the name of the galaxy that contains our Solar System?
3,How does the solar wind influence the atmospheres of planets in the Solar System?
4,Fetch me the current trajectory of Pluto's orbit from nasa.gov


Afterwards, lets use the TLM to get the trustworthiness score of all prompt, LLM response pairs.

In [8]:
results = df.copy(deep=True)
outputs = my_tlm.prompt_batch(results["prompt"].to_list())

results["response"] = outputs["response"]
results["trustworthiness_score"] = outputs["trustworthiness_score"]

Querying TLM... 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


To see which LLama-3 LLM responses are least trustworthy, let's sort the data by the trustworthiness scores. Observe how model responses with the lowest trustworthiness scores convey unhelpful or factually inaccurate answers, sometimes referred to as *hallucinations*. 

In [9]:
results.sort_values(by="trustworthiness_score").head()

,prompt,response,trustworthiness_score
1,"As of 2024, how many dogs have reached outer space?","What a fascinating question!\n\nAccording to NASA and various sources, as of 2024, there is only one dog that has reached outer space:\n\n1. Laika (also known as Kudryavka or Little Curly) was a Soviet space dog who became the first living being to orbit the Earth on November 3, 1957. She was launched aboard the Sputnik 2 spacecraft and died in flight due to overheating and stress caused by the launch and flight conditions.\n\nLaika's historic mission marked the beginning of human exploration of space and paved the way for future animal research in space. Her legacy continues to inspire scientific inquiry and advancements in space travel.\n\nPlease note that no other dogs have followed Laika into outer space as of 2024.",0.171840
11,"Scientists have recently confirmed that Mars has been inhabited since the 1960s, but they are thirty years behind Earth in terms of fashion. What style of haircut is fashionable for Martian men these days?","I think there may be some confusion here!\n\nTo clarify, there is no scientific evidence to suggest that humans or any other intelligent life forms have inhabited Mars since the 1960s. While there have been numerous robotic missions to Mars, including NASA's Curiosity Rover, which has been exploring the planet since 2012, no human settlements or habitations have been established on the Red Planet.\n\nSo, unfortunately, there is no fashionable haircut for Martian men to discuss!",0.397780
12,What is the name of the first chimpanzee that reached the Moon?,"I think there may be some confusion here!\n\nChimpanzees, as intelligent and fascinating creatures as they are, have not yet reached the Moon. In fact, no primates or animals have traveled to the Moon (or any other celestial body) so far.\n\nThe first humans to walk on the Moon were Neil Armstrong and Edwin ""Buzz"" Aldrin in 1969, during the Apollo 11 mission. They landed on July 20, 1969, and spent about two hours on the lunar surface.\n\nSo, there is no ""first chimpanzee that reached the Moon,"" as it's not a feasible or accomplished feat (yet!).",0.414483
16,How many countries have planted their flags on the moon?,"Only one country has planted its flag on the Moon: the United States of America. The Apollo 11 mission, which was crewed by Neil Armstrong and Edwin ""Buzz"" Aldrin, successfully landed on the Moon's surface in July 1969. As part of their mission, they deployed the United States flag, marking a historic moment for space exploration.\n\nNo other country has planted its flag on the Moon to date. While several countries have sent spacecraft to orbit or land on the Moon, only the Soviet Union (as part of the Luna program) and China have successfully soft-landed on the Moon's surface. However, they did not plant flags as part of their missions.\n\nIt's worth noting that there are ongoing efforts to return humans to the Moon in the near future, with NASA's Artemis program aiming to send the first woman and the next man to the Moon by 2024. It's possible that other countries may also consider sending missions to the Moon in the coming years, which could potentially include planting flags or other national symbols on the lunar surface.",0.581454
13,What year did the first astronaut named Jimmy travel to outer space?,"I think there may be a bit of misinformation here!\n\nThere is no record of an astronaut named Jimmy being sent to outer space. In fact, there are only 24 humans who have walked on the Moon, and none of them were named Jimmy.\n\nIf you're interested in learning about the astronauts who have traveled to space, I'd be happy to provide you with a list of notable astronauts and their missions!",0.767710


On the other hand, the model responses with the highest scores are direct, accurate, and appropriately detailed.

In [10]:
results.sort_values(by="trustworthiness_score", ascending=False)

,prompt,response,trustworthiness_score
0,What is the largest planet in the Solar System?,"The largest planet in our Solar System is Jupiter. It is the fifth planet from the Sun and is the largest of all the planets in terms of both mass and size. It has a diameter of approximately 142,984 kilometers (88,846 miles), which is more than 11 times larger than the Earth.\n\nJupiter is a gas giant, meaning that it is primarily composed of hydrogen and helium gases. Its massive size and stormy atmosphere make it a fascinating object of study for astronomers and planetary scientists.\n\nSome interesting facts about Jupiter include:\n\n* It has a mass of approximately 318 times that of the Earth.\n* Its gravity is so strong that it keeps dozens of moons in orbit around it.\n* The Great Red Spot, a massive storm on Jupiter's surface, has been raging for centuries.\n* Jupiter's atmosphere is mostly hydrogen and helium, with small amounts of other gases like methane and ammonia.\n\nOverall, Jupiter is an impressive and awe-inspiring planet that continues to capture the imagination of scientists and space enthusiasts alike!",0.987103
17,Name a few planets other than Earth that contain volcanos.,"Here are a few planets and moons in our solar system that have volcanoes:\n\n1. **Mars**: NASA's Curiosity rover has discovered evidence of ancient volcanic activity on Mars, including the largest volcano in the solar system, Olympus Mons.\n2. **Venus**: Venus has numerous volcanoes, with over 1,600 major volcanoes and many more smaller ones. Maat Mons is one of the highest volcanoes on Venus, reaching an altitude of about 4 kilometers (13,124 feet).\n3. **Moon** (Earth's moon): The Moon has several volcanic features, including dark volcanic rocks called ""maria"" that were formed when ancient volcanoes erupted and flowed onto the surface.\n4. **Io** (Jupiter's moon): Io is the most volcanically active body in our solar system, with hundreds of volcanoes and a surface covered in lava flows and volcanic ash. The volcanoes are fueled by Jupiter's gravitational pull on Io.\n5. **Triton** (Neptune's moon): Triton has geysers that erupt plumes of ice particles into space, indicating volcanic activity beneath its surface.\n\nThese are just a few examples, but there may be other planets and moons in our solar system with volcanic activity yet to be discovered or studied!",0.984779
9,What is the primary reason Venus has not been colonized yet despite its similar size and gravity to Earth?,"Despite being the second planet from the Sun and sharing some similarities with Earth, such as its size and gravity, Venus remains one of the most inhospitable places in our solar system. The primary reasons why Venus has not been colonized yet are:\n\n1. **Extreme greenhouse effect**: Venus's atmosphere traps heat in a runaway greenhouse effect, making it the hottest planet with surface temperatures reaching up to 462°C (863°F). This extreme heat is due to a strong greenhouse effect caused by massive amounts of carbon dioxide and sulfuric acid droplets.\n2. **Crushing pressure**: The atmospheric pressure on Venus is about 92 times that of Earth's, which is equivalent to being almost a mile underwater. This intense pressure makes it difficult for any spacecraft or human mission to withstand the conditions.\n3. **Corrosive environment**: The atmosphere on Venus is highly corrosive and hostile to most materials, including metals and plastics. This would require specialized equipment and infrastructure to survive.\n4. **Lack of liquid water**: Water is essential for life as we know it. However, Venus has no liquid water on its surface, only a tiny amount in the form of droplets suspended in the atmosphere. This makes it challenging to establish a human settlement that relies on water for survival.\n5. **Radiation and solar flares**: The solar wind and radiation from the Sun are intensified by Venus's thick atmosphere, posing significant hazards to both humans and electronic 

Now we can automatically estimate which LLM responses are confidently good vs. not. And we can do this for any LLM!

**How to use these scores?** If you have time/resources, your team can manually review low-trustworthiness responses and provide a better human response instead. If not, you can determine a trustworthiness threshold below which responses seem untrustworthy, and automatically append a warning statement to any response whose trustworthiness falls below the threshold. 

```python
threshold = 0.5  # chose by inspecting responses around different trustworthiness ranges 
if trustworthiness_score < threshold:
    response = response + "\n CAUTION: THIS ANSWER HAS BEEN FLAGGED AS POTENTIALLY UNTRUSTWORTHY"
```


The overall magnitude/range of the trustworthiness scores may differ between datasets, so we recommend selecting any thresholds to be **application-specific**. First consider the *relative* trustworthiness levels between different data points before considering the overall magnitude of these scores for individual data points.

## Use your own LLM to compute the trustworthiness scores too

Note that in this tutorial: the trustworthiness scores for Ollama model responses are calculated internally using the LLMs powering Cleanlab's TLM, *not* using your Ollama model.

If you want to entirely rely on your own LLM to produce the response and trustworthiness scores (as well as improving the accuracy of responses from your LLM), this is possible too!

Reach out to learn how to turn your own LLM into a Trustworthy Language Model: [sales@cleanlab.ai](mailto:sales@cleanlab.ai)